In [1]:
import struct
from tqdm import tqdm_notebook
from sklearn import svm, model_selection, metrics

In [2]:
def decoding_mnist_rawData(dataStyle='train', maxCount=0):
    label_f = open(f'../data/mnist/{dataStyle}-labels-idx1-ubyte','rb')
    image_f = open(f'../data/mnist/{dataStyle}-images-idx3-ubyte','rb')
    csv_f = open(f'../data/mnist/{dataStyle}.csv', 'w', encoding='utf-8')
    label_magic_number, label_count = struct.unpack('>II',label_f.read(4+4))
    image_magic_number, image_count, row, col = struct.unpack('>IIII',image_f.read(4+4+4+4))

    pixels = row * col

    for idx in tqdm_notebook(range(image_count)):
        if idx >= maxCount: break
        label_tmp = struct.unpack('B', label_f.read(1))
        label = label_tmp[0]

        binaryData = image_f.read(pixels)
        strData = list(map(lambda x : str(x), binaryData))
        csv_f.write(str(label)+',')
        csv_f.write(','.join(strData)+'\n')

        with open(f'../data/mnist/{label}.pgm','w',encoding='utf-8') as f:
            f.write('P2 28 28 255\n' + ' '.join(strData))
        
    label_f.close()
    image_f.close()
    csv_f.close()

In [3]:
def load_csv(dataType='train'):
    f = open(f'../data/mnist/{dataType}.csv', 'r')
    
    labels = list()
    images = list()
    
    while True:
        row = f.readline()
        if not row: break
        labels.append(int(row.split(',')[0]))
        images.append(list(map(lambda x: int(x)/256, row.split(',')[1:])))
    f.close()
    return { 'labels':labels, 'images':images }

In [7]:
result_ac_scores = list()
result_clf_reports = list()
for cnt in range(1,5):
    decoding_mnist_rawData('t10k',2500*cnt)
    decoding_mnist_rawData('train',7500*cnt)
    train = load_csv('train')
    test = load_csv('t10k')
    SEED = 0
    clf = svm.SVC(random_state=SEED)
    clf.fit(train['images'], train['labels'])
    predict = clf.predict(test['images'])
    ac_score = metrics.accuracy_score(test['labels'], predict)
    print(f'테스트셋 {250*cnt}개일 때 정확도:',ac_score)
    result_ac_scores.append(ac_score)
    clf_report = metrics.classification_report(test['labels'], predict)
    result_clf_reports.append(clf_report)


테스트셋 250개일 때 정확도: 0.9344


테스트셋 500개일 때 정확도: 0.9522


테스트셋 750개일 때 정확도: 0.9670666666666666


테스트셋 1000개일 때 정확도: 0.9742
